<a href="https://colab.research.google.com/github/Yuva2832/FMML_LAB_PROJECT/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

EXERCISE:1 Reducing Variance: When you use a single split of your data for training and validation, the validation accuracy can vary significantly depending on how the data was split. Some splits might be easier or harder for the model to learn from. By averaging the results across multiple splits, you reduce the impact of any one split that might have been particularly easy or difficult, giving you a better estimate of the model's general performance.Better Generalization: Averaging results across multiple splits (like in cross-validation) ensures that every data point is used for both training and validation at some point. This comprehensive evaluation can provide a more accurate reflection of the model's ability to generalize to unseen data.Mitigating Bias: Some splits might have an unintended bias due to the random nature of the splitting process or the underlying distribution of the data. By averaging across multiple splits, you mitigate the effect of such biases, leading to more robust evaluation metrics.Statistical Stability: Multiple splits lead to multiple measurements of accuracy, allowing you to calculate not just an average but also variance or standard deviation, providing a sense of the statistical stability of your results.

EXERCISE:2. Yes, averaging validation accuracy across multiple splits generally provides a more accurate estimate of the test accuracy. Here's why:

1. **Comprehensive Use of Data**: When using multiple splits (such as in k-fold cross-validation), each data point gets a chance to be in both the training and validation sets. This means the model is evaluated on every data point, leading to a more comprehensive assessment of its performance. As a result, the averaged validation accuracy is more likely to reflect how the model will perform on unseen test data.

2. **Reduced Overfitting to Specific Splits**: A single train-validation split might result in a model that is inadvertently tuned to the specifics of that split, which can lead to an overly optimistic or pessimistic estimate of performance. By using multiple splits and averaging, you minimize the risk of overfitting to any particular split, making the estimate of test accuracy more reliable.

3. **Balanced Evaluation**: Multiple splits ensure that the model's performance is evaluated under various conditions and data distributions. This helps in providing a balanced view of the model's robustness and effectiveness, leading to a more accurate estimation of how it will perform in real-world scenarios.

4. **Variance Estimation**: Multiple splits allow for calculation of variance in validation accuracy, providing insight into the stability of the model's performance. If the variance is low, it suggests that the model performs consistently, which is a good indicator that the average accuracy is a reliable estimate of test accuracy.

EXERCISE:3  Effect of Increasing the Number of IterationsReduced Variance in Estimates: As you increase the number of iterations, the variance in your validation accuracy estimates generally decreases. This means that your estimate of model performance becomes more stable and less sensitive to the specific random splits of the data. With more iterations, you average out the effects of any unusual splits, leading to a more reliable estimate.Better Estimation of Test Accuracy: Higher iterations provide a better estimate of test accuracy because the model is evaluated on more variations of the training and validation sets. This comprehensive evaluation reduces the likelihood that the estimate is biased due to a particular data partition. For example, in k-fold cross-validation, using more folds (like 10 instead of 5) means each validation fold is smaller, and each training fold is closer in size to the full dataset, often leading to a more precise estimate of test accuracy.Greater Representation of Data Distribution: With more iterations, more combinations of the dataset are used for training and validation. This thorough use of the data helps ensure that the model's performance estimate better represents the true underlying data distribution, leading to more generalizable results.Diminishing Returns with More IterationsHowever, there are diminishing returns:Computational Cost: Increasing the number of iterations also increases computational cost because the model has to be trained and validated multiple times. Beyond a certain point, the additional accuracy or stability gained by increasing iterations may not justify the extra computational resources required.Marginal Gains: After a certain number of iterations (often around 10 in k-fold cross-validation), the marginal gains in the precision of the estimate diminish. For example, moving from 5 to 10 iterations may significantly reduce variance in the estimate, but moving from 10 to 20 might not offer a proportionate improvement.Overhead in Model Training: In some cases, particularly with very large datasets or complex models, the increased number of iterations might lead to significant training overhead, which could be impractical depending on time and resource constraints.

EXERCISE:4 Benefits of Increasing Iterations with Small DatasetsMaximizing Data Usage: When you have a small dataset, using methods like cross-validation with more iterations ensures that every data point is used for both training and validation. This maximizes the use of the limited data you have, helping to better estimate the model's performance and make the most of the available data.Reduced Overfitting to Specific Splits: With more iterations, the model is trained and validated on multiple different splits of the data, reducing the risk of overfitting to any specific split. This is particularly important when dealing with small datasets, where individual splits can have a significant impact on model performance due to limited data diversity.More Robust Performance Estimates: Higher iterations help in getting a more stable and robust estimate of model performance because they average out the variability caused by small validation sets. With small datasets, each data point's inclusion or exclusion can have a large effect, so multiple iterations help balance this out.Limitations and ConsiderationsInherent Data Limitations: Even with many iterations, a very small dataset has inherent limitations that cannot be fully overcome. A small training dataset means the model may not learn all the necessary patterns, and a small validation set means performance estimates might still be noisy or biased. More iterations can reduce some of this variability, but they can't add new information to the dataset.Model Generalizability: If the dataset is too small, increasing iterations cannot compensate for a lack of data diversity and representativeness. The model may learn patterns that are too specific to the small dataset and not generalize well to unseen data. More iterations do not change the fact that the model has seen the same data points repeatedly, which can lead to overfitting or overly optimistic performance estimates.Computational Cost vs. Benefit: With a very small dataset, increasing iterations might provide diminishing returns. If the dataset size is already limited, the benefits of adding more iterations in terms of performance estimation accuracy will decrease. There comes a point where the computational cost and time required to run many iterations outweigh the marginal gains in estimation accuracy.Validation Accuracy Variability: While more iterations might provide a more stable average estimate of model performance, the variability of validation accuracy might remain high due to the small number of data points. High variability indicates that the model's performance is still not reliably estimated.